In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\PC\\Text_summarization_final\\research'

In [3]:
os.chdir('../')

In [4]:
pwd

'c:\\Users\\PC\\Text_summarization_final'

In [7]:
from dataclasses import dataclass
from pathlib import Path

In [10]:
@dataclass(frozen=True)
class DataTransfomrationConfig:
     root_dir: Path
     data_path: Path
     tokenizer_name:Path


In [9]:
from textSummarization.constant import *
from textSummarization.utils.common import read_yaml,create_directories

In [16]:
class ConfrigurationManager:
     def __init__(
          self,
          config_filepath=CONFIG_FILE_PATH,
          params_filepath=PARAMS_FILE_PATH):
          self.config=read_yaml(config_filepath)
          self.params=read_yaml(params_filepath)
          create_directories([self.config.artifacts_root])
     
     def get_data_transformation_config(self)->DataTransfomrationConfig:
          config=self.config.data_transformation
          create_directories([config.root_dir])
          # print("----")
          data_transformation_config=DataTransfomrationConfig(
               root_dir= config.root_dir,
               data_path=config.data_path,
               tokenizer_name=config.tokenizer_name,
          )
          
          return data_transformation_config

In [15]:
import os 
from textSummarization.logging import logger 
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

c:\Users\PC\Text_summarization_final\senv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-02-09 17:38:27,080:INFO:config:PyTorch version 2.2.0 available.]


In [19]:
class DataTransformation:
    def __init__(self, config: DataTransfomrationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self,example_batch):
        input_encodings=self.tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings=self.tokenizer(example_batch['summary'],max_length=128,truncation=True)
            
        return {
            'inputs_ids':input_encodings['input_ids'],
            'attention_mask':input_encodings['attention_mask'],
            'labels':target_encodings['input_ids']
        }
        
        
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
            
        
     


    
    
    

In [20]:
config = ConfrigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.convert()

[2024-02-09 17:55:24,625:INFO:common:yaml file :config\config.yaml loaded success ]
[2024-02-09 17:55:24,650:INFO:common:yaml file :params.yaml loaded success ]
[2024-02-09 17:55:24,652:INFO:common:created directory at: artifacts]
[2024-02-09 17:55:24,657:INFO:common:created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\PC\Text_summarization_final\senv\lib\site-packages\transformers\tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 61368.71 examples/s]
